# Advent of code 2021: Day 5. Hydrothermal Venture

- Daniel Cerdán Vélez.
- Sunday, December 5, 2021

In [1]:
import math
import numpy as np
import pandas as pd
import re

### --- Day 5: Hydrothermal Venture ---

You come across a field of [hydrothermal vents](https://en.wikipedia.org/wiki/Hydrothermal_vent) on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in **lines**; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format `x1,y1 -> x2,y2` where `x1`,`y1` are the coordinates of one end the line segment and `x2`,`y2` are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like `1,1 -> 1,3` covers points `1,1`, `1,2`, and `1,3`.
- An entry like `9,7 -> 7,7` covers points `9,7`, `8,7`, and `7,7`.

For now, **only consider horizontal and vertical lines**: lines where either `x1 = x2` or `y1 = y2`.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is `0,0` and the bottom right corner is `9,9`. Each position is shown as **the number of lines which cover that point** or `.` if no line covers that point. The top-left pair of `1`s, for example, comes from `2,2 -> 2,1`; the very bottom row is formed by the overlapping lines `0,9 -> 5,9` and `0,9 -> 2,9`.

To avoid the most dangerous areas, you need to determine **the number of points where at least two lines overlap**. In the above example, this is anywhere in the diagram with a `2` or larger - a total of **`5`** points.

Consider only horizontal and vertical lines. **At how many points do at least two lines overlap?**

In [2]:
# I'm going to create the diagram automatically by going through the file once. Initially I have selected
# enough random dimensions (1000 x 1000), but it would not work if the numbers are greater.
pattern = "[0-9]*,[0-9]* -> [0-9]*,[0-9]*"

with open("../data/input_05.txt", "r") as f:
    lines = f.readlines()
    lines = [line.rstrip() for line in lines]

aux = np.matrix([re.split(",| -> ", item) for item in lines]).astype(int)

x_max = max(np.max(aux[:,0]),np.max(aux[:,2]))
y_max = max(np.max(aux[:,1]),np.max(aux[:,3]))

In [3]:
diagram = np.full((y_max+1,x_max+1),0)

with open("../data/input_05.txt", "r") as f:
    for line in f:
        
        start, stop = map(lambda x: x.split(','), line.strip().split(' -> '))
        start = [int(item) for item in start]
        stop = [int(item) for item in stop]
        
        if (int(start[0]) == int(stop[0])) or (int(start[1]) == int(stop[1])):
            (x1, y1), (x2, y2) = sorted([start, stop])
            diagram[y1:y2+1, x1:x2+1] += 1

result = np.sum(diagram > 1)

print("Result =\033[1m", result,"\033[0m")

Result = 7468 


### --- Part Two ---

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider **diagonal lines**.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like `1,1 -> 3,3` covers points `1,1`, `2,2`, and `3,3`.
- An entry like `9,7 -> 7,9` covers points `9,7`, `8,8`, and `7,9`.

Considering all lines from the above example would now produce the following diagram:
```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```
You still need to determine **the number of points where at least two lines overlap**. In the above example, this is still anywhere in the diagram with a `2` or larger - now a total of **`12`** points.

Consider all of the lines. **At how many points do at least two lines overlap?**


In [4]:
diagram = np.full((y_max+1,x_max+1),0)

with open("../data/input_05.txt", "r") as f:
    for line in f:
        
        start, stop = map(lambda x: x.split(','), line.strip().split(' -> '))
        start = [int(item) for item in start]
        stop = [int(item) for item in stop]
        
        if (stop[0]-start[0]) == 0:
            slope = 0
        else:
            slope = (stop[1]-start[1])/(stop[0]-start[0])
            
        angle = math.degrees(math.atan(slope))
        
        if ((int(start[0]) == int(stop[0])) or (int(start[1]) == int(stop[1])) or abs(angle) == 45):
            (x1, y1), (x2, y2) = sorted([start, stop])
            if angle == 0:
                diagram[int(y1):int(y2)+1, int(x1):int(x2)+1] += 1
            elif angle == 45:
                for i, j in zip(range(y1, y2+1), range(x1, x2+1)):
                    diagram[i,j] += 1
            else: # angle == -45
                for i, j in zip(range(y1, y2-1, -1), range(x1, x2+1)):
                    diagram[i,j] += 1
            

result = np.sum(diagram > 1)

print("Result =\033[1m", result,"\033[0m")

Result = 22364 
